In [ ]:
import json
import pandas as pd
from collections import defaultdict, Counter
with open("outputs/buckets_08-04_manual-llm-manual.json","r") as file:
  buckets = json.load(file)
with open("outputs/extra_buckets_08-04.json","r") as file:
  extra_buckets = json.load(file)




datasets = [f"data/US/{term*2+2009-222}-{term*2+2010-222}_{term}th_Congress/csv/history.csv" for term in range(111,120)]
history_df_by_term=[pd.read_csv(ds) for ds in datasets]
for i,df in enumerate(history_df_by_term):
  df["term"]=i+111
history_df=pd.concat(history_df_by_term)


bucket_map={}
for name in buckets:
  for action in buckets[name]:
    bucket_map[action]=name
extra_bucket_map=defaultdict(list)
n_read_twice=0
for name in extra_buckets:
  for i,action in enumerate(extra_buckets[name]):
    extra_bucket_map[action].append(name)
bucket_lens=Counter()
for action in history_df["action"]:
  bucket_lens[bucket_map[action]]+=1
extra_bucket_lens=Counter()
for i,action in enumerate(history_df["action"]):
  for extra_bucket in extra_bucket_map[action]:
    extra_bucket_lens[extra_bucket]+=1
common_bucket_names = [name for name in buckets.keys() if bucket_lens[name]>=50]
common_extra_bucket_names = [name for name in extra_buckets.keys() if extra_bucket_lens[name]>=50]
common_bucket_names_inv={name:i for i,name in enumerate(common_bucket_names)}
common_extra_bucket_names_inv={name:i for i,name in enumerate(common_extra_bucket_names)}
# with open("./outputs/bucket_names_output.jsonl", "r") as file:
#   for line in file:
#     if line.strip():  # Skip empty lines
#       response = json.loads(line)
#       llm_buckets.append(response["response"]["body"]["output"][0]["content"][0]["text"])
      
# history_df["llm_bucket"] = history_df["bucket"].map(dict(zip(bucket_names, llm_buckets)))
# display(bucket_map)
unbucketed=[]
for action in history_df["action"]:
  if action not in bucket_map:
    unbucketed.append(action)
# print(len(unbucketed))
# Add next bill_id column to compare
history_df["bucket"]=history_df["action"].apply(lambda action:bucket_map[action])
history_df["extra_buckets"]=history_df["action"].apply(lambda action:extra_bucket_map[action])
bills={bill_id:group for (bill_id,group) in history_df.groupby("bill_id")}
bill_ids = list(bills.keys())

MIN_TERM=min(history_df["term"])
MAX_TERM=max(history_df["term"])
N_TERMS=MAX_TERM-MIN_TERM+1


In [ ]:
display(len(common_bucket_names))
display(len(common_extra_bucket_names))

In [ ]:
display(sum(extra_bucket_lens[name] for name in common_extra_bucket_names))

In [ ]:
import numpy as np

alpha=2/3
data=[]

bill_id=history_df.iloc[0]["bill_id"]
predecessors_buckets=np.zeros(len(common_bucket_names))
predecessor_buckets=np.zeros(len(common_bucket_names))
predecessors_extra_buckets=np.zeros(len(common_extra_bucket_names))
predecessor_extra_buckets=np.zeros(len(common_extra_bucket_names))
curr_bucket = np.zeros(len(common_bucket_names))
curr_extra_buckets = np.zeros(len(common_extra_bucket_names))
inputs=[]
outputs=[]
n_predecessors_prev=0
cum_extra_buckets=0
for i,row in history_df.iterrows():
  prev_bucket=np.array(curr_bucket)
  prev_extra_buckets=np.array(curr_extra_buckets)
  curr_bucket = np.zeros(len(common_bucket_names))
  curr_extra_buckets = np.zeros(len(common_extra_bucket_names))
  output_bucket = np.zeros(len(common_bucket_names)+1)
  if row["bucket"] in common_bucket_names_inv:
    curr_bucket[common_bucket_names_inv[row.bucket]]=1
    output_bucket[common_bucket_names_inv[row.bucket]]=1
  else:
    output_bucket[-1]=1
  for extra_bucket in row["extra_buckets"]:
    # print("extra bucket:",extra_bucket)
    if extra_bucket in common_extra_bucket_names_inv:
      curr_extra_buckets[common_extra_bucket_names_inv[extra_bucket]]=1
  s=sum(curr_extra_buckets)
  # print(s)
  cum_extra_buckets+=s
  
  chamber = np.zeros(2)
  if row["chamber"]=="House":
    chamber[0]=1
  if row["chamber"]=="Senate":
    chamber[1]=1

  # if sum(one_hot_prev)!=1 and sum(one_hot_prev)!=0:
  #   print("non-0/1 sum",i)
  # n_predecessors=np.sum(predecessors_vec)
  # if n_predecessors!=0 and n_predecessors!=n_predecessors_prev+1 and n_predecessors!=n_predecessors_prev:
  #   print(n_predecessors,n_predecessors_prev,i)
  # n_predecessors_prev=n_predecessors
  if row["bill_id"]!=bill_id:
    bill_id=row["bill_id"]
    prev_bucket=np.zeros(len(common_bucket_names))
    prev_extra_buckets=np.zeros(len(common_extra_bucket_names))
    predecessors_buckets=np.zeros(len(common_bucket_names))
    predecessor_buckets=np.zeros(len(common_bucket_names))
    predecessors_extra_buckets=np.zeros(len(common_extra_bucket_names))
    predecessor_extra_buckets=np.zeros(len(common_extra_bucket_names))
  predecessor_buckets=(1-alpha)*predecessor_buckets+alpha*np.array(prev_bucket)
  predecessors_buckets=np.array(predecessors_buckets)+prev_bucket
  predecessor_extra_buckets=(1-alpha)*predecessor_extra_buckets+alpha*np.array(prev_extra_buckets)
  predecessors_extra_buckets=np.array(predecessors_extra_buckets)+prev_extra_buckets
  term=np.zeros(N_TERMS)
  term[row.term-MIN_TERM]=1
  # print(predecessor_bu/ckets)
  # input_vec = np.concat([predecessor_vec,predecessors_vec,one_hot_term])
  entry={"predecessor_buckets":predecessor_buckets,"predecessors_buckets":predecessors_buckets,"predecessor_extra_buckets":predecessor_extra_buckets,"predecessors_extra_buckets":predecessors_extra_buckets,"term":term,"chamber":chamber,
         "output_bucket":output_bucket,"output_extra_buckets":curr_extra_buckets} # print(entry)
  data.append(entry)

print(cum_extra_buckets)
# print(data[0])
# inputs=np.stack(inputs)
# outputs=np.stack(outputs)
# np.savetxt("outputs/vectors/07-29_charsim17/input",inputs[0])
# np.savetxt("outputs/vectors/07-29_charsim17/output",outputs)


In [ ]:
print([entry for entry in data])

In [ ]:
display(sum([sum(entry["output_extra_buckets"]) for entry in data]))

In [ ]:
display(cum_extra_buckets)

In [ ]:
print(cum_extra_buckets)

In [ ]:
np.savez_compressed("outputs/prediction_vecs_08-04_withextras.npz",data)

In [ ]:
print(extra_bucket_lens)

In [ ]:
import numpy as np
data = np.load("outputs/prediction_vecs_08-04_withextras.npz",allow_pickle=True)["arr_0"]
display(data[1])
# display(outs)

In [ ]:
display([x for x in data if "predecessor_buckets" not in x])

In [ ]:
import sklearn.preprocessing
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import sklearn

class ActionDataset(Dataset):
  def __init__(self,path):
    data = np.load(path,allow_pickle=True)["arr_0"]
    self.inputs = [np.concatenate([entry["predecessor_buckets"],entry["predecessors_buckets"],entry["predecessor_extra_buckets"],entry["predecessors_extra_buckets"],entry["term"],entry["chamber"]]) for entry in data]
    self.inputs = sklearn.preprocessing.StandardScaler().fit_transform(self.inputs)
    self.outputs = [np.concatenate((entry["output_bucket"],entry["output_extra_buckets"])) for entry in data]
  def __len__(self):
    return len(self.inputs)
  def __getitem__(self,idx):
    return self.inputs[idx],self.outputs[idx]

ds = ActionDataset("outputs/prediction_vecs_08-04_withextras.npz")
# print(len(ds))
train_dataset,test_dataset,val_dataset = torch.utils.data.random_split(ds,[0.6,0.2,0.2])

In [ ]:
def print_arr(arr):
  for line in arr:
    print([float(x) for x in line])

In [ ]:
torch.set_grad_enabled(True)

In [ ]:
import torch
import os
from torch.utils.data import DataLoader
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
val_loader = DataLoader(val_dataset,batch_size=32,shuffle=True)

# class ActionModel(torch.nn.Module):
#   def __init__(self,input_len,output_len):
#     self.linear= torch.nn.Linear
#     self.input_len=input_len
#     self.output_len = output_len
#   def forward(self,input):

model = torch.nn.Linear(len(common_bucket_names)*2+len(common_extra_bucket_names)*2+N_TERMS+2,len(common_bucket_names)+1+len(common_extra_bucket_names))
# model.load_state_dict(torch.load("outputs/models/08-04_lr1e-5_beta.3/epoch85.pt"))
optim = torch.optim.Adam(model.parameters(),lr=1e-5)
pred_loss_fn = torch.nn.BCEWithLogitsLoss()
beta=1e-4
folder = "outputs/models/08-04_withextras_bce_lr1e-5_beta1e-4"
if not os.path.isdir(folder):
    os.mkdir(folder)

for epoch in range(200):
    for i,(input,output) in enumerate(train_loader):
        optim.zero_grad()
        input=input.float()
        output=output.float()
        pred = model(input)
        pred_loss = pred_loss_fn(pred,output)
        # print(pred_loss.requires_grad)
        ridge_loss = torch.norm(model.weight,p=1)
        loss = pred_loss+beta*ridge_loss
        loss.backward()
        optim.step()
    print(f"finished epoch {epoch}")
    if epoch<5 or (epoch%5==0 and epoch<=20) or epoch%10==0:
        torch.save({"model":model.state_dict(),"optim":optim.state_dict()},folder+f"/epoch{epoch}.pt")
    with torch.no_grad():
        val_pred_loss=torch.scalar_tensor(0)
        val_ridge_loss=torch.sum(torch.abs(model.weight))
        for input,output in val_loader:
            input=input.float()
            output=output.float()
            pred = model(input)
            # for line in output:
            #   for i,x in enumerate(line):
            #     if x>=0.9:
            #       print(i)
            #   print(torch.sum(line))
            # break
            # print_arr(output.numpy())
            # print("sum:",torch.sum(output))
            val_pred_loss+=pred_loss_fn(pred,output)
        val_pred_loss/=len(val_loader)
    print("log val pred loss:",torch.log10(val_pred_loss),"val ridge loss:",val_ridge_loss)
    with open(folder+f"/loss","a") as file:
        file.write(f"epoch {epoch}. val pred loss:{val_pred_loss}. val ridge loss:{val_ridge_loss}")
        file.write("\n")



In [ ]:
import torch
model = torch.nn.Linear(len(common_bucket_names)*2+len(common_extra_bucket_names)*2+N_TERMS+2,len(common_bucket_names)+1+len(common_extra_bucket_names))
model.load_state_dict(torch.load("outputs/models/08-04_withextras_bce_lr1e-5_beta1e-4/epoch200.pt")["model"])
val_loader = DataLoader(val_dataset,batch_size=32,shuffle=True)
# val_loader = DataLoader(val_dataset,batch_size=32,shuffle=True)
import itertools
def argmax(l):

  return l.index(max(l))
    
with torch.no_grad():
  val_pred_loss=torch.scalar_tensor(0)
  val_ridge_loss=torch.sum(torch.abs(model.weight))
  eps=1e-2
  for input,output in val_loader:
    input=input.float()
    output=output.float()
    pred = model(input)
    for input_line,output_line,pred_line in zip(input,output,pred):
      # print(len(common_bucket_names))
      # print(line[-1])
      # line=list(line)
      # output_dict = {name:torch.sigmoid(logodds) for name,logodds in zip(common_bucket_names+["No Bucket"]+common_extra_bucket_names,pred_line)}
      # if torch.sum(output_line[-len(common_extra_bucket_names):])>0:
      # print(list(output_line).index(1))
        # errors = sorted([((float(torch.sigmoid(logodds)-output_p)),float(output_p),name) for name,output_p,logodds in zip(common_bucket_names+["No Bucket"]+["Extra bucket: "+x for x in common_extra_bucket_names],output_line,pred_line) if torch.abs(output_p-1)<eps],reverse=True)
      # print([float(x) for x in output_line[-len(common_extra_bucket_names):]])
      # print(output_line[:-len(common_extra_bucket_names)])
      # break
        # display(errors)
      
    # break# for i,x in enumerate(line):
      #   if x>=0.1:
      #     print(i,float(x),end="")
      # print()
      # print(torch.sum(line))
    # break
    # print_arr(output.numpy())
    # print("sum:",torch.sum(output))
  # val_pred_loss/=len(val_loader)
  # print("val pred loss:",val_pred_loss,"val ridge loss:",val_ridge_loss)

In [ ]:
print(common_bucket_names)

In [ ]:
from collections import defaultdict
import torch
import itertools
# model.load_state_dict(torch.load("outputs/models/08-04_lr1e-5_beta.01/epoch99.pt")["model"])
model = torch.nn.Linear(len(common_bucket_names)*2+len(common_extra_bucket_names)*2+N_TERMS+2,len(common_bucket_names)+1+len(common_extra_bucket_names))
model.load_state_dict(torch.load("outputs/models/08-04_withextras_bce_lr1e-5_beta1e-06/epoch200.pt")["model"])

weights = model.weight.detach().numpy()
predecessor_chains=[]
predecessors_chains=[]
predecessor_s_chains=[]
predecessor_plus_s_chains=[]
predecessor_no_s_chains=[]
term_chains=[]
common_extra_bucket_names_prefixed=["Extra Bucket: "+x for x in common_extra_bucket_names]
for i,bucket1 in enumerate(common_bucket_names):
  for j,bucket2 in enumerate(itertools.chain(common_bucket_names,common_extra_bucket_names_prefixed)):
    # print(i)
    predecessor_chains.append((float(weights[j][i]),bucket1,bucket2))
    predecessors_chains.append((float(weights[j][i+len(common_bucket_names)]),bucket1,bucket2))
    predecessor_plus_s_chains.append((predecessors_chains[-1][0]+predecessor_chains[-1][0],bucket1,bucket2))
    predecessor_no_s_chains.append((predecessor_chains[-1][0]-predecessors_chains[-1][0],bucket1,bucket2))
    predecessor_s_chains.append((predecessors_chains[-1][0]-predecessor_chains[-1][0],bucket1,bucket2))
for i,bucket1 in enumerate(common_extra_bucket_names_prefixed):
  for j,bucket2 in enumerate(common_bucket_names+common_extra_bucket_names_prefixed):
    predecessor_chains.append((float(weights[j][i+2*len(common_bucket_names)]),bucket1,bucket2))
    predecessors_chains.append((float(weights[j][i+2*len(common_bucket_names)+len(common_extra_bucket_names)]),bucket1,bucket2))
    predecessor_plus_s_chains.append((predecessors_chains[-1][0]+predecessor_chains[-1][0],bucket1,bucket2))
    predecessor_no_s_chains.append((predecessor_chains[-1][0]-predecessors_chains[-1][0],bucket1,bucket2))
    predecessor_s_chains.append((predecessors_chains[-1][0]-predecessor_chains[-1][0],bucket1,bucket2))
for term in range(N_TERMS):
  for j,bucket2 in enumerate(itertools.chain(common_bucket_names,common_extra_bucket_names_prefixed)):
    term_chains.append((float(weights[j][term+2*len(common_bucket_names)+2*len(common_extra_bucket_names)]),str(term*2+2009)+"-"+str(term*2+2010),bucket2))
chamber_chains=[]
for j,bucket in enumerate(itertools.chain(common_bucket_names,common_extra_bucket_names_prefixed)):
    chamber_chains.append((float(weights[j][1+term+2*len(common_bucket_names)+2*len(common_extra_bucket_names)]),"House",bucket))
for j,bucket in enumerate(itertools.chain(common_bucket_names,common_extra_bucket_names_prefixed)):
    chamber_chains.append((float(weights[j][2+term+2*len(common_bucket_names)+2*len(common_extra_bucket_names)]),"Senate",bucket))

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np

def bucket_by_magnitude(chains):
  return Counter([np.round(np.log(abs(x[0]))) for x in chains])

# display(predecessor_effects)
term_effects = bucket_by_magnitude(predecessor_no_s_chains)
plt.bar(term_effects.keys(),term_effects.values())
plt.yscale("log")

In [ ]:
print(predecessor_effects)

In [ ]:
def display_chains(chains):
  display([str(param[0])+" "+param[1]+" -> "+param[2] for param in sorted(chains,key=lambda x:abs(x[0]),reverse=True) if param[1]!=param[2] or param[0]>0][:100])
  print(sum(x[0] for x in sorted(chains,key=lambda x: abs(x[0]),reverse=True)[-10:])/10)
display_chains(predecessor_chains)
# for param in sorted(predecessor_s_chains,key=lambda x: abs(x[0]),reverse=True)[:100]:
#   print(param[0],param[1],"->",param[2])
# for param in sorted(predecessor_chains,key=lambda x: abs(x[0]),reverse=True)[-100:]:
#   print(param[0],param[1],"->",param[2])
# for param in sorted(predecessor_chains[-100:],key=lambda x: abs(x[0]),reverse=True):
#   print(param[0],param[1],"->",param[2])

In [ ]:
# Sklearn
import sklearn
import copy
import random
model = sklearn.linear_model.LogisticRegression(penalty="l1",solver="saga",multi_class="multinomial")

scaler = sklearn.preprocessing.StandardScaler()
data_shuffled = copy.deepcopy(data)
random.shuffle(data_shuffled)
inputs = [np.concatenate([e["predecessors"],e["predecessor"],e["term"]]) for e in data_shuffled]
outputs = [e["output"] for e in data_shuffled]
inputs= scaler.fit_transform(inputs)
divider = int(len(data)*0.8)
train_inputs,test_inputs = inputs[:divider], data_shuffled[divider:]
train_outputs,test_outputs = outputs[:divider], outputs[divider:]
model.fit(train_inputs,train_outputs)
test_pred = model.predict_proba(test_inputs) 